# Script that sets up training repository

Change the settings in the first cell and run all to create the repository and send the invitations.

In [ ]:
import os, sys, json,importlib, base64,urllib, time

#This notebook requires you to install https://github.com/PyGithub/PyGithub first
from github import Github, InputGitAuthor

#Add parent folder to library paths
module_path = os.path.abspath(os.path.join('../../Common-Resources'))
if module_path not in sys.path:
    sys.path.append(module_path)
import github_tools as ghTools

In [ ]:
accessToken = ghTools.getTokenAttribute(attribute="token")
ghSession = Github(accessToken)

#get user
user = ghSession.get_user()
username = user.login

### Manually input name of repository

In [ ]:
#The name of the repository to create, use random 4 letters to get unique url
reponame = "pr-training"

#Variables if repo should be created on org account
dime_training_account = "dime-wb-trainings"
use_org_account = True

Switch host to org account

In [ ]:
# Set the account where the training repo will be created
if use_org_account:
    account = ghSession.get_organization(dime_training_account)
else:
    account = user

## Creating the repo

In [ ]:
#making sure the repo is not already tere
try:
    del_repo = ghSession.get_repo(account.login+"/"+reponame)
    print("Deleted: " + del_repo.full_name)
    del_repo.delete()
except:
    print("No repo with that name")

In [ ]:
#Create the repo
training_repo = account.create_repo(reponame, auto_init=True)
print(training_repo)
print(training_repo.html_url)

### Replace the auto initialized repo with the prepared README and add .gitignore

#### update README

In [ ]:
#Get the commit sha from where the README was created in the auto_init
contents = training_repo.get_contents("README.md")

#Read the README file in this repo to get the content for the commit
file_content = open('repo-content-files/README.md', 'rb').read()
#print(file_content)

#Update the README file
readme_update_result = training_repo.update_file("README.md", 
                                                 "Update README.md", 
                                                 file_content,
                                                 contents.sha,
                                                 branch="main")

readme_update_sha = readme_update_result["commit"].sha
print(readme_update_result)

#### add .gitignore

In [ ]:
#Read the README file in this repo to get the content for the commit
file_content = open("repo-content-files/no.gitignore", 'rb').read()

#Update the README file
create_ignore_result = training_repo.create_file(".gitignore", "add a ignore file", file_content, branch="main")

### Start adding lyrics files

In [ ]:
list_of_files = [
    ["lyrics/pop/backstreets-back.txt","Adding Backstreet Back - Go BSB!"],
    ["lyrics/rock/stairway-to-heaven.txt","Stairway to Heaven! So GOOOD."],
    ["lyrics/rock/november-rain.txt","November Rain - The solo is AMAZING!"],
    ["lyrics/pop/mamma-mia.txt","ABBA is the best! And this song is their best!"]
]

for song_list in list_of_files:
    #Read the README file in this repo to get the content for the commit
    file_content = open("repo-content-files/" + song_list[0], 'rb').read()
    #print(file_content)

    #Update the README file
    create_result = training_repo.create_file(song_list[0], song_list[1], file_content, branch="main")
    
last_sha = create_result["commit"].sha
print(last_sha)

### Edit in other branch

In [ ]:
#Create the improve lyrics branch
pr_branch = "improve-lyrics"
training_repo.create_git_ref("refs/heads/" + pr_branch, last_sha)

In [ ]:
list_of_files = [
    ["november-rain1.txt","rock/november-rain.txt","I like April rains better"],
    ["november-rain2.txt","rock/november-rain.txt","One chorus in the end is enough"],
    ["backstreets-back.txt","pop/backstreets-back.txt","I like April rains better"],
    ["november-rain3.txt","rock/november-rain.txt","Axel Rose singing Eros Ramazzotti "],
]

for song_list in list_of_files:
    #Read the file in this repo to get the content for the commit
    file_content = open("repo-content-files/edits/" + song_list[0], 'rb').read()

    #Update the file
    content = training_repo.get_contents("lyrics/"+song_list[1],ref="refs/heads/"+pr_branch)
    create_result = training_repo.update_file("lyrics/"+song_list[1],song_list[2],file_content,content.sha,branch=pr_branch)
    
    last_commit = create_result["commit"]
    


### Create PR

In [ ]:
base = "main"
head = pr_branch

title = "Making improvements to these lyrics"
body = """In meeting on June 12th, 2021 the DIME Lyrics team decided that I will improve these lyrics.

We decided to mostly focus on November Rain and Backstreet Back."""


pr = training_repo.create_pull(base=base,head=head,title=title,body=body)

print(pr)

In [ ]:
overall_comment = "Thank you @kbjarkefur! I ran this code. I was able to reproduce your work. However, it would be good if you can improve the documentation, so it is easer to find the master file that runs all other files."
pr.create_issue_comment(overall_comment)